In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re

In [2]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv(os.path.abspath('../data/Processed/Cities_imputed.csv'), index_col=0)

In [4]:
climate_df = df[['City', 'Country', 'Continent']]

In [5]:
climate_df['wiki_link'] = [re.sub(r'\([^)]*\)', '', city).replace(" ", "_") for city in climate_df.City]

In [6]:
climate_df.loc[climate_df['wiki_link'] == 'Birmingham']

,City,Country,Continent,wiki_link
319,Birmingham(AL),United States,North America,Birmingham
261,Birmingham,United Kingdom,Europe,Birmingham


In [7]:
climate_df.loc[climate_df.index == 278] 

,City,Country,Continent,wiki_link
278,San Francisco Bay Area(CA),United States,North America,San_Francisco_Bay_Area


In [8]:
def update_wiki_link(df, city, city_update):
    df.loc[df['wiki_link'] == city, 'wiki_link'] = city_update

In [9]:
update_wiki_link(climate_df, 'Santa_Cruz', 'Santa_Cruz_(Bolivia)')

In [10]:
cities_updates = [
    ('Santa_Cruz', 'Santa_Cruz_(Bolivia)'),
    ('Cordoba' , 'Córdoba,_Argentina'),
    ('San_Francisco_Bay_Area', 'San_Fransisco'),
    ('Tampa-St._Petersburg', 'Saint_Petersburg_(Florida)'),
    ('Memphis', 'Memphis,_Tennessee')
]

for city, city_update in cities_updates:
    update_wiki_link(
        df = climate_df,
        city = city,
        city_update = city_update
    )

    ('Birmingham', 'Birmingham,_Alabama')
    ('Valencia', 'Valencia,_Carabobo'),

In [11]:
climate_df.loc[(climate_df['wiki_link'] == 'Birmingham') & (climate_df['Country'] == 'United States')] = 'Birmingham,_Alabama'
climate_df.loc[(climate_df['wiki_link'] == 'Valencia') & (climate_df['Country'] == 'United Venezuela')] = 'Valencia,_Carabobo'

In [12]:
climate_df

,City,Country,Continent,wiki_link
284,Baltimore(MD),United States,North America,Baltimore
9,Melbourne,Australia,Oceania,Melbourne
185,Niamey,Niger,Africa,Niamey
327,Hanoi,Vietnam,Asia,Hanoi
66,Urumqi,China,Asia,Urumqi
...,...,...,...,...
225,Durban,South Africa,Africa,Durban
103,Kumasi,Ghana,Africa,Kumasi
250,Izmir,Turkey,Asia,Izmir
249,Ankara,Turkey,Asia,Ankara


In [234]:
def seperate_metrix_from_imperial(row, th_data):
    rainfall = ['Average precipitation mm (inches)', 'Average rainfall inches (mm)', 'Average rainy days', 'Average precipitation inches (mm)', 'Average rainfall mm (inches)']

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            # if rainfall when select minimeters
            if th_data in rainfall:
                matric.append(max(val1, val2))
            # else it is a temperature select celcius
            else:
                matric.append(min(val1, val2))
        except ValueError:
            matric.append(float(imperial_metric))
    return matric

In [235]:
wiki_link = 'Medellin'
url = 'https://en.wikipedia.org/wiki/' + wiki_link

response = requests.get(url)
status_code = response.status_code
print(status_code)

soup = BeautifulSoup(response.text,"html.parser")


200


In [236]:
tables = soup.findAll('table',{"class":"collapsible"})
for table in tables:
    if table.findAll(['th'])[0].getText()[0:12].lower() == 'climate data':
        break
values = []
trs = table.findAll(['tr'])[1:]


In [237]:
focus_climate_data = [
    'Record high °C (°F)', 'Record high °F (°C)',
    'Average high °C (°F)', 'Average high °F (°C)', 
    'Daily mean °C (°F)', 'Daily mean °F (°C)', 'Daily mean °C (°F)',
    'Average low °C (°F)', 'Average low °F (°C)',
    'Record low °C (°F)', 'Record low °F (°C)',
    'Average precipitation mm (inches)', 'Average rainfall inches (mm)', 'Average rainy days', 'Average precipitation inches (mm)', 'Average rainfall mm (inches)',
    'Average precipitation days (≥ 0.1 mm)', 'Average rainy days (≥ 0.01 in)', 'Average precipitation days (≥ 1.0 mm)', 'Average precipitation days (≥ 0.5 mm)', 'Average precipitation days (≥ 0.01 in)', 'Average rainy days (≥ 0.1 mm)'
]

climate_data_names = [
    'Record high °C (°F)', 
    'Average high °C (°F)', 
    'Daily mean °C (°F)',
    'Average low °C (°F)', 
    'Record low °C (°F)', 
    'Average precipitation mm (inches)', 
    'Average precipitation days (≥ 0.1 mm)',
]

In [238]:

for i, cell in enumerate(trs):

    th = cell.find_all('th')
    th_data = [col.text.strip('\n') for col in th]  
    td = cell.find_all('td')
    row = [i.text.replace('\n','') for i in td]
    
    
    if i == 0:
        columns = th_data
        city_climate_df_tmp = pd.DataFrame(columns=columns)
    else:
        
        # exstract the metrix represeantion of the climate data
        if len(th_data) == 0:
            continue
        if th_data[0] in focus_climate_data:
            metric_row = seperate_metrix_from_imperial(row, th_data[0])

            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+metric_row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)

In [240]:
city_climate_df_tmp['Month'] = climate_data_names